_* These examples are created by Deb the DBA for her blog post, Working with Synonyms._

## Scenario One:
Create synonym for non-existing object 

1. Confirm none of the objects exist:

In [2]:
IF EXISTS (SELECT * FROM sys.synonyms WHERE name = 'NonexistingObject')
DROP SYNONYM NonexistingObject;

IF EXISTS (SELECT * FROM sys.objects WHERE name = 'NonexistingObject')
DROP TABLE NonexistingObject;
GO

Commands completed successfully.

Total execution time: 00:00:00.015

2. Create the synonym for the non existing table:

In [3]:
CREATE SYNONYM NonexistingObject FOR dbo.NonexistingObject;

Commands completed successfully.

Total execution time: 00:00:00.005

3. Select from the synonym:

In [4]:
SELECT * FROM NonexistingObject;

: Msg 470, Level 16, State 1, Line 1
The synonym "NonexistingObject" referenced synonym "dbo.NonexistingObject". Synonym chaining is not allowed.

Total execution time: 00:00:00.005

4. Now create the underlying table:

In [5]:
CREATE TABLE dbo.NonexistingObject
	(PrimaryKeyCol	int IDENTITY(1,1) NOT NULL
		CONSTRAINT pk_NonexistingObject PRIMARY KEY CLUSTERED,
	FakeColumnOne		varchar(10)	NOT NULL,
	AnotherFakeColumn	varchar(10)	NULL
	);
GO


: Msg 2714, Level 16, State 6, Line 1
There is already an object named 'NonexistingObject' in the database.

Total execution time: 00:00:00.007

You can't have two objects in the same schema with the same name - even if they're different object types. So let's reset for our second scenario,

In [6]:
IF EXISTS (SELECT * FROM sys.synonyms WHERE name = 'NonexistingObject')
DROP SYNONYM NonexistingObject;

IF EXISTS (SELECT * FROM sys.objects WHERE name = 'NonexistingObject')
DROP TABLE NonexistingObject;
GO

Commands completed successfully.

Total execution time: 00:00:00.022

## Scenario Two:
Same as the first but setting up with different names

1. Confirm that the objcts don't exist

In [7]:
IF EXISTS (SELECT * FROM sys.objects WHERE name = 'NonexistingTable')
DROP TABLE dbo.NonexistingTable;

IF EXISTS (SELECT * FROM sys.synonyms WHERE name = 'NonexistingSynonym')
DROP SYNONYM NonexistingSynonym;

Commands completed successfully.

Total execution time: 00:00:00.022

2. Create synonym for non existing table:

In [8]:
IF NOT EXISTS (SELECT * FROM sys.synonyms WHERE name = 'NonexistingSynonym')
CREATE SYNONYM NonexistingSynonym FOR dbo.NonexistingTable;

Commands completed successfully.

Total execution time: 00:00:00.011

3. Select from the synonym:

In [9]:
SELECT * FROM NonexistingSynonym;

: Msg 5313, Level 16, State 1, Line 1
Synonym 'NonexistingSynonym' refers to an invalid object.

Total execution time: 00:00:00.002

4. Now create the table:

In [10]:
CREATE TABLE dbo.NonexistingTable
	(PrimaryKeyCol	int IDENTITY(1,1) NOT NULL
		CONSTRAINT pk_NonexistingTable PRIMARY KEY CLUSTERED,
	FakeColumnOne		varchar(10)	NOT NULL,
	AnotherFakeColumn	varchar(10)	NULL
	)
;

Commands completed successfully.

Total execution time: 00:00:00.005

5. Select from the synonym:

In [11]:
SELECT * FROM NonexistingSynonym;

(0 rows affected)

Total execution time: 00:00:00.077

PrimaryKeyCol,FakeColumnOne,AnotherFakeColumn


6. Now what happens if we modify the table:

In [12]:
ALTER TABLE dbo.NonexistingTable
ADD YetAnotherColumn	varchar(10) NULL;

Commands completed successfully.

Total execution time: 00:00:00.092

7. Select from the synonym:

In [13]:
SELECT * FROM NonexistingSynonym;

(0 rows affected)

Total execution time: 00:00:00.008

PrimaryKeyCol,FakeColumnOne,AnotherFakeColumn,YetAnotherColumn


8. Now we can clean up:

In [14]:
IF EXISTS (SELECT * FROM sys.objects WHERE name = 'NonexistingTable')
DROP TABLE dbo.NonexistingTable;

IF EXISTS (SELECT * FROM sys.synonyms WHERE name = 'NonexistingSynonym')
DROP SYNONYM NonexistingSynonym;

Commands completed successfully.

Total execution time: 00:00:00.048

## Scenario Three:
How does this behavior affect other related objects, like views and procedures? 

1. Let's make sure none of the objects exist first:

In [15]:
IF EXISTS (SELECT * FROM sys.objects WHERE Name = 'TableStillNotHere')
DROP TABLE TableStillNotHere
IF EXISTS (SELECT * FROM sys.objects WHERE Name = 'vStillNotHere')
DROP VIEW vStillNotHere
IF EXISTS (SELECT * FROM sys.objects WHERE Name = 'pr_StillNotHere')
DROP PROC pr_StillNotHere
IF EXISTS (SELECT * FROM sys.synonyms WHERE name = 'StillNotHere')
DROP SYNONYM StillNotHere


Commands completed successfully.

Total execution time: 00:00:00.033

2. Create the synonym for the non-existant table

In [16]:
CREATE SYNONYM StillNotHere for dbo.TableStillNotHere;

Commands completed successfully.

Total execution time: 00:00:00.001

3. Create the stored proc referencing the synonym:

In [17]:
CREATE PROCEDURE pr_StillNotHere 
	@FilterMe bit
AS
	SELECT * 
	FROM StillNotHere
	WHERE isDataProperlyFiltered = @FilterMe;
GO

Commands completed successfully.

Total execution time: 00:00:00.004

4. Create the veiw over the synonym:

In [18]:
CREATE VIEW vStillNotHere
AS 
SELECT *
FROM StillNotHere
WHERE isDataProperlyFiltered = 1;
GO

: Msg 5313, Level 16, State 1, Procedure vStillNotHere, Line 4
Synonym 'StillNotHere' refers to an invalid object.

Total execution time: 00:00:00.015

5. Create the table so we can create the view:

In [19]:
CREATE TABLE dbo.TableStillNotHere
	(ID	int IDENTITY(1,1) NOT NULL
		CONSTRAINT PK_TableStillNotHere PRIMARY KEY CLUSTERED,
	isDataProperlyFiltered bit NOT NULL
	);
GO

Commands completed successfully.

Total execution time: 00:00:00.004

6. Now create the view:

In [20]:
CREATE VIEW vStillNotHere
AS 
SELECT *
FROM StillNotHere
WHERE isDataProperlyFiltered = 1
GO

Commands completed successfully.

Total execution time: 00:00:00.002

7. Test the view by SELECTing from it:

In [21]:
SELECT * FROM vStillNotHere;

(0 rows affected)

Total execution time: 00:00:00.005

ID,isDataProperlyFiltered


8. Now let's modify the table to see what happens:

In [22]:
ALTER TABLE TableStillNotHere
ADD NewColumn int ;
GO

Commands completed successfully.

Total execution time: 00:00:00.005

9. Is there a difference between the view and the table?

In [23]:
SELECT * FROM vStillNotHere;
SELECT * FROM StillNotHere;
GO

(0 rows affected)

(0 rows affected)

Total execution time: 00:00:00.010

ID,isDataProperlyFiltered


ID,isDataProperlyFiltered,NewColumn


10. Refresh the view and then redo your SELECT:

In [24]:
EXEC sp_refreshview vStillNotHere;
GO

SELECT * FROM vStillNotHere;
GO

Commands completed successfully.

(0 rows affected)

Total execution time: 00:00:00.036

ID,isDataProperlyFiltered,NewColumn


11. Let's modify that column to change the data type:

In [25]:
ALTER TABLE TableStillNotHere
ALTER COLUMN NewColumn varchar(20);
GO

Commands completed successfully.

Total execution time: 00:00:00.012

12. Now let's look at the meta data for the change:

In [26]:
SELECT TABLE_NAME, COLUMN_NAME, DATA_TYPE 
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME IN ('TableStillNotHere', 'vStillNotHere')
AND COLUMN_NAME = 'NewColumn';

(2 rows affected)

Total execution time: 00:00:00.060

TABLE_NAME,COLUMN_NAME,DATA_TYPE
TableStillNotHere,NewColumn,varchar
vStillNotHere,NewColumn,int


13. What happens when we insert data:

In [27]:
INSERT INTO TableStillNotHere (isDataProperlyFiltered, NewColumn)
VALUES (1, 'Varchar data');

SELECT * FROM vStillNotHere;

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.024

ID,isDataProperlyFiltered,NewColumn
1,1,Varchar data


14. Now that we've cleared that up, let's clean up our test objects.

In [28]:
-- cleanup
IF EXISTS (SELECT * FROM sys.objects WHERE Name = 'TableStillNotHere')
DROP TABLE TableStillNotHere;

IF EXISTS (SELECT * FROM sys.objects WHERE Name = 'vStillNotHere')
DROP VIEW vStillNotHere;

IF EXISTS (SELECT * FROM sys.objects WHERE Name = 'pr_StillNotHere')
DROP PROC pr_StillNotHere;

IF EXISTS (SELECT * FROM sys.synonyms WHERE name = 'StillNotHere')
DROP SYNONYM StillNotHere;

Commands completed successfully.

Total execution time: 00:00:00.155